#Import files and read in as dataframes

In [1]:
# reading in files from google drive to speed up run time
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

##Crashes
https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
Crash data shows information about each traffic crash on city streets within the City of Chicago limits and under the jurisdiction of Chicago Police Department (CPD). Data are shown as is from the electronic crash reporting system (E-Crash) at CPD, excluding any personally identifiable information. Records are added to the data portal when a crash report is finalized or when amendments are made to an existing report in E-Crash. Data from E-Crash are available for some police districts in 2015, but citywide data are not available until September 2017. About half of all crash reports, mostly minor crashes, are self-reported at the police district by the driver(s) involved and the other half are recorded at the scene by the police officer responding to the crash. Many of the crash parameters, including street condition data, weather condition, and posted speed limits, are recorded by the reporting officer based on best available information at the time, but many of these may disagree with posted information or other assessments on road conditions. If any new or updated information on a crash is received, the reporting officer may amend the crash report at a later time. A traffic crash within the city limits for which CPD is not the responding police agency, typically crashes on interstate highways, freeway ramps, and on local roads along the City boundary, are excluded from this dataset.

In [3]:
crashes_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_Crashes.csv")

##People
https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d
This data contains information about people involved in a crash and if any injuries were sustained. This dataset should be used in combination with the traffic Crash and Vehicle dataset. Each record corresponds to an occupant in a vehicle listed in the Crash dataset. Some people involved in a crash may not have been an occupant in a motor vehicle, but may have been a pedestrian, bicyclist, or using another non-motor vehicle mode of transportation. Injuries reported are reported by the responding police officer. Fatalities that occur after the initial reports are typically updated in these records up to 30 days after the date of the crash. Person data can be linked with the Crash and Vehicle dataset using the “CRASH_RECORD_ID” field. A vehicle can have multiple occupants and hence have a one to many relationship between Vehicle and Person dataset. However, a pedestrian is a “unit” by itself and have a one to one relationship between the Vehicle and Person table.

In [4]:
people_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_People.csv")

<ipython-input-4-8866f2488915>:1: DtypeWarning: Columns (20,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  people_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_People.csv")


In [5]:
# replacing space in column name with underscore
people_df.rename(columns={"BAC_RESULT VALUE": "BAC_RESULT_VALUE"}, inplace=True)

##Vehicles
https://data.cityofchicago.org/Transportation/Traffic-Crashes-Vehicles/68nd-jvt3

This dataset contains information about vehicles (or units as they are identified in crash reports) involved in a traffic crash. This dataset should be used in conjunction with the traffic Crash and People dataset available in the portal. “Vehicle” information includes motor vehicle and non-motor vehicle modes of transportation, such as bicycles and pedestrians. Each mode of transportation involved in a crash is a “unit” and get one entry here. Each vehicle, each pedestrian, each motorcyclist, and each bicyclist is considered an independent unit that can have a trajectory separate from the other units. However, people inside a vehicle including the driver do not have a trajectory separate from the vehicle in which they are travelling and hence only the vehicle they are travelling in get any entry here. This type of identification of “units” is needed to determine how each movement affected the crash. Data for occupants who do not make up an independent unit, typically drivers and passengers, are available in the People table. Many of the fields are coded to denote the type and location of damage on the vehicle. Vehicle information can be linked back to Crash data using the “CRASH_RECORD_ID” field. Since this dataset is a combination of vehicles, pedestrians, and pedal cyclists not all columns are applicable to each record. Look at the Unit Type field to determine what additional data may be available for that record.


In [6]:
vehicles_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_Vehicles.csv")

<ipython-input-6-20edec7b5672>:1: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Phase3-Project-Data/Traffic_Crashes_-_Vehicles.csv")


#Creating master dataset

1. Keep only the columns we want
2. clean data and trouble shoot (match data types, consistent formatting, etc)
3. merge into one data set
4. Filter by date: 2018 - 2023 (keep in mind city budgeting season begins in June)

###Crashes dataframe edit

In [7]:
selected_columns = ['CRASH_RECORD_ID', 'CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'POSTED_SPEED_LIMIT','WEATHER_CONDITION',
                    'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE', 'MOST_SEVERE_INJURY',
                    'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'LOCATION']
crashes_edit_df = crashes_df[selected_columns].copy()

In [8]:
# forcing datetime
crashes_edit_df['CRASH_DATE'] = pd.to_datetime(crashes_edit_df['CRASH_DATE'], errors='coerce' )

In [30]:
crashes_edit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739105 entries, 0 to 739104
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          739105 non-null  object        
 1   CRASH_DATE               739105 non-null  datetime64[ns]
 2   CRASH_HOUR               739105 non-null  int64         
 3   CRASH_DAY_OF_WEEK        739105 non-null  int64         
 4   CRASH_MONTH              739105 non-null  int64         
 5   POSTED_SPEED_LIMIT       739105 non-null  int64         
 6   WEATHER_CONDITION        739105 non-null  object        
 7   LIGHTING_CONDITION       739105 non-null  object        
 8   ROADWAY_SURFACE_COND     739105 non-null  object        
 9   ROAD_DEFECT              739105 non-null  object        
 10  CRASH_TYPE               739105 non-null  object        
 11  HIT_AND_RUN_I            230382 non-null  object        
 12  PRIM_CONTRIBUTOR

In [9]:
# filtering by date
crashes_edit_df_2018_2023 = crashes_edit_df[(crashes_edit_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [31]:
crashes_edit_df_2018_2023.sort_values(by='CRASH_DATE')

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,LOCATION
111894,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,0,2,1,25,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.641013916869 41.681735373094)
631253,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,2018-01-01 00:00:00,0,2,1,20,CLEAR,DAYLIGHT,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.589225075066 41.80057461981)
209356,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,0,2,1,30,CLEAR,UNKNOWN,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.65026813462 41.955842185147)
573304,dfb3229940c672426c4010d7348d16b69b030191213b86...,2018-01-01 00:05:00,0,2,1,30,CLEAR,"DARKNESS, LIGHTED ROAD",UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.66877431033 41.949417412921)
40841,821ee3270b2c503fd0905415f674d3f4a63e1ac15d9100...,2018-01-01 00:07:00,0,2,1,35,SNOW,"DARKNESS, LIGHTED ROAD",SNOW OR SLUSH,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,WEATHER,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.622993129041 41.683945885086)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700084,19c606950b068ff6e704f68bcd42ac5379e3142850951e...,2023-07-06 22:20:00,22,5,7,30,CLEAR,DUSK,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,IMPROPER LANE USAGE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.765601346906 41.880344977442)
705198,a1f562015cbc6bfecec083ee97dad0cca3d9f770682764...,2023-07-06 22:45:00,22,5,7,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,IMPROPER TURNING/NO SIGNAL,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.633803891217 41.875532652069)
708813,16ec144e4a39707b2d7c51fd924bc53dff5894de75d085...,2023-07-07 00:17:00,0,6,7,25,CLEAR,DARKNESS,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.705160226138 41.843575982562)
708775,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,CLEAR,DAWN,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,FAILING TO YIELD RIGHT-OF-WAY,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.763330444094 41.892623022942)


###People dataframe edit

In [10]:
selected_columns = ['PERSON_ID', 'CRASH_RECORD_ID', 'VEHICLE_ID', 'CRASH_DATE', 'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
                    'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT', 'BAC_RESULT_VALUE']
people_edit_df = people_df[selected_columns].copy()

In [11]:
# Create 'is_driver' column in the 'people' dataframe
people_edit_df['is_driver'] = (people_edit_df['PERSON_TYPE'] == 'DRIVER').astype(int)

# Create 'total_drivers_in_crash' column by grouping by 'CRASH_RECORD_ID' and summing 'is_driver'
people_edit_df['total_drivers_in_crash'] = people_edit_df.groupby('CRASH_RECORD_ID')['is_driver'].transform('sum')

In [12]:
# forcing datetime
people_edit_df['CRASH_DATE'] = pd.to_datetime(people_edit_df['CRASH_DATE'], errors='coerce' )

In [13]:
people_edit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620956 entries, 0 to 1620955
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   PERSON_ID               1620956 non-null  object        
 1   CRASH_RECORD_ID         1620956 non-null  object        
 2   VEHICLE_ID              1589096 non-null  float64       
 3   CRASH_DATE              1620956 non-null  datetime64[ns]
 4   PERSON_TYPE             1620956 non-null  object        
 5   SEX                     1595043 non-null  object        
 6   AGE                     1148419 non-null  float64       
 7   HOSPITAL                275410 non-null   object        
 8   INJURY_CLASSIFICATION   1620267 non-null  object        
 9   DRIVER_ACTION           1289959 non-null  object        
 10  DRIVER_VISION           1289491 non-null  object        
 11  PHYSICAL_CONDITION      1290851 non-null  object        
 12  BAC_RESULT    

In [14]:
# filtering by date
people_edit_df_2018_2023 = people_edit_df[(people_edit_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [28]:
people_edit_df_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1318659 entries, 0 to 1620955
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   PERSON_ID               1318659 non-null  object        
 1   CRASH_RECORD_ID         1318659 non-null  object        
 2   VEHICLE_ID              1291471 non-null  float64       
 3   CRASH_DATE              1318659 non-null  datetime64[ns]
 4   PERSON_TYPE             1318659 non-null  object        
 5   SEX                     1296738 non-null  object        
 6   AGE                     936778 non-null   float64       
 7   HOSPITAL                231395 non-null   object        
 8   INJURY_CLASSIFICATION   1318202 non-null  object        
 9   DRIVER_ACTION           1046639 non-null  object        
 10  DRIVER_VISION           1046214 non-null  object        
 11  PHYSICAL_CONDITION      1047428 non-null  object        
 12  BAC_RESULT    

In [29]:
people_edit_df_2018_2023.sort_values(by='CRASH_DATE')

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,PERSON_TYPE,SEX,AGE,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash
703337,O294415,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,285514.0,2018-01-01 00:00:00,DRIVER,X,NaN,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1
703406,O294495,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,285601.0,2018-01-01 00:00:00,DRIVER,X,NaN,DNA,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1
703194,O294262,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,285396.0,2018-01-01 00:00:00,DRIVER,X,NaN,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1
1314562,P64097,dfb3229940c672426c4010d7348d16b69b030191213b86...,285043.0,2018-01-01 00:05:00,PASSENGER,M,NaN,REFUSED,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,0,1
702919,O293914,dfb3229940c672426c4010d7348d16b69b030191213b86...,285043.0,2018-01-01 00:05:00,DRIVER,M,53.0,REFUSED,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558021,O1612807,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,1534732.0,2023-07-07 00:19:00,DRIVER,F,29.0,NaN,NO INDICATION OF INJURY,FAILED TO YIELD,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1,2
1558022,O1612808,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,1534737.0,2023-07-07 00:19:00,DRIVER,M,19.0,NaN,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1,2
1558638,P356740,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,1534737.0,2023-07-07 00:19:00,PASSENGER,M,22.0,NaN,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,0,2
1558024,O1612817,c449bb5ee04a25261ec126d7ff1ee42cafce8e11ed9230...,1534744.0,2023-07-07 00:30:00,DRIVER,M,NaN,NaN,NO INDICATION OF INJURY,OTHER,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1,1


###Vehicles dataframe edit

In [15]:
selected_columns = ['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'CRASH_DATE','UNIT_TYPE', 'VEHICLE_USE','VEHICLE_DEFECT',
                    'TRAVEL_DIRECTION','MANEUVER','OCCUPANT_CNT','EXCEED_SPEED_LIMIT_I',]
vehicles_edit_df = vehicles_df[selected_columns].copy()

In [16]:
# forcing datetime
vehicles_edit_df['CRASH_DATE'] = pd.to_datetime(vehicles_edit_df['CRASH_DATE'], errors='coerce' )

In [24]:
vehicles_edit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1506571 entries, 0 to 1506570
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   CRASH_UNIT_ID         1506571 non-null  object        
 1   CRASH_RECORD_ID       1506571 non-null  object        
 2   CRASH_DATE            1506571 non-null  datetime64[ns]
 3   UNIT_TYPE             1504541 non-null  object        
 4   VEHICLE_USE           1472537 non-null  object        
 5   VEHICLE_DEFECT        1472537 non-null  object        
 6   TRAVEL_DIRECTION      1472537 non-null  object        
 7   MANEUVER              1472537 non-null  object        
 8   OCCUPANT_CNT          1472537 non-null  float64       
 9   EXCEED_SPEED_LIMIT_I  2397 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 114.9+ MB


In [17]:
# filtering by date
vehicles_edit_df_2018_2023 = vehicles_edit_df[(vehicles_edit_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [26]:
vehicles_edit_df_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1227534 entries, 0 to 1506570
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   CRASH_UNIT_ID         1227534 non-null  int64         
 1   CRASH_RECORD_ID       1227534 non-null  object        
 2   CRASH_DATE            1227534 non-null  datetime64[ns]
 3   UNIT_TYPE             1225896 non-null  object        
 4   VEHICLE_USE           1198573 non-null  object        
 5   VEHICLE_DEFECT        1198573 non-null  object        
 6   TRAVEL_DIRECTION      1198573 non-null  object        
 7   MANEUVER              1198573 non-null  object        
 8   OCCUPANT_CNT          1198573 non-null  float64       
 9   EXCEED_SPEED_LIMIT_I  1370 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 103.0+ MB


In [25]:
vehicles_edit_df_2018_2023.sort_values(by='CRASH_DATE')

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_TYPE,VEHICLE_USE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I
351165,294495,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,DRIVER,UNKNOWN/NA,UNKNOWN,E,UNKNOWN/NA,1.0,NaN
351079,294416,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,PARKED,PERSONAL,UNKNOWN,W,PARKED,0.0,NaN
351078,294415,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,DRIVER,UNKNOWN/NA,UNKNOWN,UNKNOWN,UNKNOWN/NA,1.0,NaN
350919,294263,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,2018-01-01 00:00:00,PARKED,PERSONAL,UNKNOWN,S,PARKED,0.0,NaN
351166,294496,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,PARKED,NOT IN USE,UNKNOWN,E,PARKED,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
71330,1612807,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,DRIVER,PERSONAL,NONE,S,STRAIGHT AHEAD,1.0,NaN
71331,1612808,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,DRIVER,PERSONAL,NONE,E,STRAIGHT AHEAD,2.0,NaN
71338,1612819,c449bb5ee04a25261ec126d7ff1ee42cafce8e11ed9230...,2023-07-07 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71337,1612818,c449bb5ee04a25261ec126d7ff1ee42cafce8e11ed9230...,2023-07-07 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Cleaning up date filtered tables for Tableau export

In [18]:
#stripping trailing and leading spaces on universal ID column
crashes_edit_df_2018_2023['CRASH_RECORD_ID'] = crashes_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()
people_edit_df_2018_2023['CRASH_RECORD_ID'] = people_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()
vehicles_edit_df_2018_2023['CRASH_RECORD_ID'] = vehicles_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()

<ipython-input-18-55f84b1e8be4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_edit_df_2018_2023['CRASH_RECORD_ID'] = crashes_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()
<ipython-input-18-55f84b1e8be4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_df_2018_2023['CRASH_RECORD_ID'] = people_edit_df_2018_2023['CRASH_RECORD_ID'].str.strip()
<ipython-input-18-55f84b1e8be4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [19]:
#Converting to string so we can deleted spaces
# Convert VEHICLE_ID in 'people' to string type
people_edit_df_2018_2023['VEHICLE_ID'] = people_edit_df_2018_2023['VEHICLE_ID'].astype(str)

# Convert CRASH_UNIT_ID in 'vehicles' to string type
vehicles_edit_df_2018_2023['CRASH_UNIT_ID'] = vehicles_edit_df_2018_2023['CRASH_UNIT_ID'].astype(str)

<ipython-input-19-9319f56662b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_df_2018_2023['VEHICLE_ID'] = people_edit_df_2018_2023['VEHICLE_ID'].astype(str)
<ipython-input-19-9319f56662b1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_edit_df_2018_2023['CRASH_UNIT_ID'] = vehicles_edit_df_2018_2023['CRASH_UNIT_ID'].astype(str)


In [20]:
#stripping trailing and leading spaces on other ID columns
people_edit_df_2018_2023['VEHICLE_ID'] = people_edit_df_2018_2023['VEHICLE_ID'].str.strip()
vehicles_edit_df_2018_2023['CRASH_UNIT_ID'] = vehicles_edit_df_2018_2023['CRASH_UNIT_ID'].str.strip()

<ipython-input-20-d0abbdd45da6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_df_2018_2023['VEHICLE_ID'] = people_edit_df_2018_2023['VEHICLE_ID'].str.strip()
<ipython-input-20-d0abbdd45da6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_edit_df_2018_2023['CRASH_UNIT_ID'] = vehicles_edit_df_2018_2023['CRASH_UNIT_ID'].str.strip()


In [21]:
# converting these back to floats as common ID's are no longer matching
people_edit_df_2018_2023['VEHICLE_ID'] = people_edit_df_2018_2023['VEHICLE_ID'].astype('float64')

vehicles_edit_df_2018_2023['CRASH_UNIT_ID'] = vehicles_edit_df_2018_2023['CRASH_UNIT_ID'].astype('float64')

<ipython-input-21-73440a4eb4cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_df_2018_2023['VEHICLE_ID'] = people_edit_df_2018_2023['VEHICLE_ID'].astype('float64')
<ipython-input-21-73440a4eb4cc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_edit_df_2018_2023['CRASH_UNIT_ID'] = vehicles_edit_df_2018_2023['CRASH_UNIT_ID'].astype('float64')


###Exporting individual table edits

In [41]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/people_edit_df_2018_2023.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  people_edit_df_2018_2023.to_csv(f)

Mounted at /content/drive


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/vehicles_edit_df_2018_2023.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  vehicles_edit_df_2018_2023.to_csv(f)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/crashes_edit_df_2018_2023.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  crashes_edit_df_2018_2023.to_csv(f)

##Cleaning up dataframes pre-merge

In [22]:
#stripping trailing and leading spaces on universal ID column
crashes_edit_df['CRASH_RECORD_ID'] = crashes_edit_df['CRASH_RECORD_ID'].str.strip()
people_edit_df['CRASH_RECORD_ID'] = people_edit_df['CRASH_RECORD_ID'].str.strip()
vehicles_edit_df['CRASH_RECORD_ID'] = vehicles_edit_df['CRASH_RECORD_ID'].str.strip()

In [23]:
# Convert VEHICLE_ID in 'people' to string type
people_edit_df['VEHICLE_ID'] = people_edit_df['VEHICLE_ID'].astype(str)

# Convert CRASH_UNIT_ID in 'vehicles' to string type
vehicles_edit_df['CRASH_UNIT_ID'] = vehicles_edit_df['CRASH_UNIT_ID'].astype(str)

In [24]:
#stripping trailing and leading spaces on other ID columns
people_edit_df['VEHICLE_ID'] = people_edit_df['VEHICLE_ID'].str.strip()
vehicles_edit_df['CRASH_UNIT_ID'] = vehicles_edit_df['CRASH_UNIT_ID'].str.strip()

In [25]:
# converting these back to floats as common ID's are no longer matching
people_edit_df['VEHICLE_ID'] = people_edit_df['VEHICLE_ID'].astype('float64')

vehicles_edit_df['CRASH_UNIT_ID'] = vehicles_edit_df['CRASH_UNIT_ID'].astype('float64')

###Checking the number of common ID's

In [26]:
# Count the number of common IDs
common_ids = people_edit_df['VEHICLE_ID'].isin(vehicles_edit_df['CRASH_UNIT_ID']).sum()
print(f"Number of common IDs: {common_ids}")

Number of common IDs: 1483147


In [27]:
# compare to pre-editing dataframe
# Count the number of common IDs
common_ids = people_df['VEHICLE_ID'].isin(vehicles_df['CRASH_UNIT_ID']).sum()
print(f"Number of common IDs: {common_ids}")

Number of common IDs: 1483147


In [28]:
# Printing ID datatypes
crashes_crash_record_id = crashes_edit_df['CRASH_RECORD_ID'].dtype
people_crash_record_id = people_edit_df['CRASH_RECORD_ID'].dtype
people_vehicle_id = people_edit_df['VEHICLE_ID'].dtype
people_person_id = people_edit_df['PERSON_ID'].dtype
vehicles_crash_unit_id = vehicles_edit_df['CRASH_UNIT_ID'].dtype
vehicles_crash_record_id = vehicles_edit_df['CRASH_RECORD_ID'].dtype

#Crashes
print(f"In crashes, crash record id data type= {crashes_crash_record_id}")

#People
print(f"In people, crash record id data type={people_crash_record_id}")
print(f"In people, vehicle id data type ={people_vehicle_id}")
print(f"In people, person id data type={people_person_id}")

#Vehicles
print(f"In vehicles, crash unit id data type= {vehicles_crash_unit_id}")
print(f"in vehicles, crash record id data type= {vehicles_crash_record_id}")

In crashes, crash record id data type= object
In people, crash record id data type=object
In people, vehicle id data type =float64
In people, person id data type=object
In vehicles, crash unit id data type= float64
in vehicles, crash record id data type= object


#Merging the 3 tables

Note: when trying to merge vehicles with people on the CRASH_UNIT_ID and VEHICLE_ID, respectively, the merge kept generating null values in records/columns where there shoud dnot be null values even after cleaning the formatting so after a few hours of troubleshooting, I abandonded that idea.

A few things I tried that all resulted in Null values for data that exists in the vehicles table:



```
df_master = pd.merge(crashes_edit_df, people_edit_df, on='CRASH_RECORD_ID', how='left')

df_master = pd.merge(df_master, vehicles_edit_df, left_on=['CRASH_RECORD_ID', 'VEHICLE_ID'], right_on=['CRASH_RECORD_ID', 'CRASH_UNIT_ID'], how='left')
```



###Upon further investigation, there are a number of records whose values (regardless of formatting and datatypes) that do not match on vehicle_id/crash_unit ID  despite what the documentation says. I am reaching out to the dataset owner. In the meantime, I am creating new individual tables for people and vehicles with only thise records that match on ID and then I shall merge again!

In [29]:
# Filter rows in the 'people' dataframe
people_edit_filtered_df = people_edit_df[people_edit_df['VEHICLE_ID'].isin(vehicles_edit_df['CRASH_UNIT_ID'])]

# Filter rows in the 'vehicles' dataframe
vehicles_edit_filtered_df = vehicles_edit_df[vehicles_edit_df['CRASH_UNIT_ID'].isin(people_edit_df['VEHICLE_ID'])]


In [112]:
people_edit_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1483147 entries, 0 to 1620955
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   PERSON_ID               1483147 non-null  object        
 1   CRASH_RECORD_ID         1483147 non-null  object        
 2   VEHICLE_ID              1483147 non-null  float64       
 3   CRASH_DATE              1483147 non-null  datetime64[ns]
 4   PERSON_TYPE             1483147 non-null  object        
 5   SEX                     1459995 non-null  object        
 6   AGE                     1046208 non-null  float64       
 7   HOSPITAL                239001 non-null   object        
 8   INJURY_CLASSIFICATION   1482820 non-null  object        
 9   DRIVER_ACTION           1177174 non-null  object        
 10  DRIVER_VISION           1177176 non-null  object        
 11  PHYSICAL_CONDITION      1177175 non-null  object        
 12  BAC_RESULT    

In [113]:
vehicles_edit_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1192166 entries, 1 to 1506493
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   CRASH_UNIT_ID         1192166 non-null  float64       
 1   CRASH_RECORD_ID       1192166 non-null  object        
 2   CRASH_DATE            1192166 non-null  datetime64[ns]
 3   UNIT_TYPE             1190502 non-null  object        
 4   VEHICLE_USE           1165269 non-null  object        
 5   VEHICLE_DEFECT        1165269 non-null  object        
 6   TRAVEL_DIRECTION      1165269 non-null  object        
 7   MANEUVER              1165269 non-null  object        
 8   OCCUPANT_CNT          1165269 non-null  float64       
 9   EXCEED_SPEED_LIMIT_I  2044 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 100.1+ MB


###Filtering by date and exporting these new filtered tables as csv

In [126]:
# filtering by date
vehicles_edit_filtered_df_2018_2023 = vehicles_edit_filtered_df[(vehicles_edit_filtered_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [127]:
# filtering by date
people_edit_filtered_df_2018_2023 = people_edit_filtered_df[(people_edit_filtered_df['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [128]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/vehicles_edit_filtered_df_2018_2023.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  vehicles_edit_filtered_df_2018_2023.to_csv(f)

In [129]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/people_edit_filtered_df_2018_2023.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  people_edit_filtered_df_2018_2023.to_csv(f)

# Going back to one crash per record with aggregate data
Note: can't seem to reconcile the duplicates.

In [67]:
# First, create 'total_vehicles_in_crash' and 'total_people_in_crash' columns
vehicles_edit_filtered_df['total_vehicles_in_crash'] = vehicles_edit_filtered_df.groupby('CRASH_RECORD_ID')['CRASH_UNIT_ID'].transform('count')
people_edit_filtered_df['total_people_in_crash'] = people_edit_filtered_df.groupby('CRASH_RECORD_ID')['PERSON_ID'].transform('count')

# Then, aggregate 'vehicles' and 'people' dataframes
vehicles_agg = vehicles_edit_filtered_df.groupby('CRASH_RECORD_ID').agg({'CRASH_UNIT_ID': 'nunique'}).reset_index()
people_agg = people_edit_filtered_df.groupby('CRASH_RECORD_ID').agg({'PERSON_ID': 'nunique'}).reset_index()

# Rename columns for clarity
vehicles_agg.rename(columns={'CRASH_UNIT_ID': 'num_unique_vehicles'}, inplace=True)
people_agg.rename(columns={'PERSON_ID': 'num_unique_people'}, inplace=True)

# Merge the aggregated dataframes with the 'crashes' dataframe
df_master_agg = pd.merge(crashes_edit_df, vehicles_agg, on='CRASH_RECORD_ID', how='left')
df_master_agg = pd.merge(df_master_agg, people_agg, on='CRASH_RECORD_ID', how='left')


<ipython-input-67-d06f77dc2a6f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_edit_filtered_df['total_vehicles_in_crash'] = vehicles_edit_filtered_df.groupby('CRASH_RECORD_ID')['CRASH_UNIT_ID'].transform('count')
<ipython-input-67-d06f77dc2a6f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_edit_filtered_df['total_people_in_crash'] = people_edit_filtered_df.groupby('CRASH_RECORD_ID')['PERSON_ID'].transform('count')


In [68]:
df_master_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739105 entries, 0 to 739104
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          739105 non-null  object        
 1   CRASH_DATE               739105 non-null  datetime64[ns]
 2   CRASH_HOUR               739105 non-null  int64         
 3   CRASH_DAY_OF_WEEK        739105 non-null  int64         
 4   CRASH_MONTH              739105 non-null  int64         
 5   POSTED_SPEED_LIMIT       739105 non-null  int64         
 6   WEATHER_CONDITION        739105 non-null  object        
 7   LIGHTING_CONDITION       739105 non-null  object        
 8   ROADWAY_SURFACE_COND     739105 non-null  object        
 9   ROAD_DEFECT              739105 non-null  object        
 10  CRASH_TYPE               739105 non-null  object        
 11  HIT_AND_RUN_I            230382 non-null  object        
 12  PRIM_CONTRIBUTOR

In [69]:
df_master_agg.head()

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,LOCATION,num_unique_vehicles,num_unique_people
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,IMPROPER TURNING/NO SIGNAL,"REPORTED, NOT EVIDENT",1.0,0.0,0.0,POINT (-87.64120093714 41.884547224337),1.0,3.0
1,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,2018-09-05 08:40:00,8,4,9,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,"VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS,...",NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.740659314632 41.968562453871),2.0,2.0
2,0115ade9a755e835255508463f7e9c4a9a0b47e9304238...,2022-07-15 00:45:00,0,6,7,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,UNKNOWN,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.716203130599 41.886336409761),2.0,2.0
3,017040c61958d2fa977c956b2bd2d6759ef7754496dc96...,2022-07-15 18:50:00,18,6,7,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.667997321599 41.925110815832),2.0,2.0
4,78eee027ec3dcc85d36c9e3fdae4729dcc56440105d65b...,2018-06-03 17:00:00,17,1,6,30,CLEAR,UNKNOWN,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.731388754145 41.910757551599),2.0,NaN


In [70]:
df_master_agg.sort_values(by='CRASH_DATE')

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,LOCATION,num_unique_vehicles,num_unique_people
431343,a802658be15312809c771559e4f81088cfb226830792a5...,2013-03-03 16:48:00,16,1,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,Y,IMPROPER LANE USAGE,"REPORTED, NOT EVIDENT",2.0,0.0,0.0,POINT (-87.708743641643 41.975856915535),1.0,1.0
17312,19fb5af681f833c2af85734245f737fa6fbe62ac1ea379...,2013-06-01 20:29:00,20,7,6,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,UNABLE TO DETERMINE,NONINCAPACITATING INJURY,1.0,0.0,0.0,POINT (-87.689968732521 42.005144534455),2.0,1.0
626040,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18 18:14:00,18,7,1,30,CLEAR,DUSK,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.638756189808 41.885609917063),2.0,2.0
388010,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21 07:40:00,7,3,1,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,FAILING TO YIELD RIGHT-OF-WAY,NONINCAPACITATING INJURY,1.0,0.0,0.0,POINT (-87.561946030143 41.760710194223),1.0,2.0
148468,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24 19:45:00,19,2,2,30,RAIN,DAYLIGHT,WET,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.701142757538 41.884016475152),2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700084,19c606950b068ff6e704f68bcd42ac5379e3142850951e...,2023-07-06 22:20:00,22,5,7,30,CLEAR,DUSK,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,IMPROPER LANE USAGE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.765601346906 41.880344977442),NaN,2.0
705198,a1f562015cbc6bfecec083ee97dad0cca3d9f770682764...,2023-07-06 22:45:00,22,5,7,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,IMPROPER TURNING/NO SIGNAL,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.633803891217 41.875532652069),NaN,4.0
708813,16ec144e4a39707b2d7c51fd924bc53dff5894de75d085...,2023-07-07 00:17:00,0,6,7,25,CLEAR,DARKNESS,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.705160226138 41.843575982562),NaN,2.0
708775,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,CLEAR,DAWN,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,FAILING TO YIELD RIGHT-OF-WAY,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.763330444094 41.892623022942),NaN,3.0


In [71]:
# filtering by date
df_master_agg_2018_2023 = df_master_agg[(df_master_agg['CRASH_DATE'] > "2017-12-31 23:59:59")]


In [73]:
df_master_agg_2018_2023.sort_values(by='CRASH_DATE')

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,HIT_AND_RUN_I,PRIM_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,LOCATION,num_unique_vehicles,num_unique_people
111894,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,0,2,1,25,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.641013916869 41.681735373094),2.0,1.0
631253,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,2018-01-01 00:00:00,0,2,1,20,CLEAR,DAYLIGHT,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.589225075066 41.80057461981),1.0,1.0
209356,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,0,2,1,30,CLEAR,UNKNOWN,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,Y,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.65026813462 41.955842185147),2.0,1.0
573304,dfb3229940c672426c4010d7348d16b69b030191213b86...,2018-01-01 00:05:00,0,2,1,30,CLEAR,"DARKNESS, LIGHTED ROAD",UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.66877431033 41.949417412921),1.0,2.0
40841,821ee3270b2c503fd0905415f674d3f4a63e1ac15d9100...,2018-01-01 00:07:00,0,2,1,35,SNOW,"DARKNESS, LIGHTED ROAD",SNOW OR SLUSH,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,WEATHER,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.622993129041 41.683945885086),2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700084,19c606950b068ff6e704f68bcd42ac5379e3142850951e...,2023-07-06 22:20:00,22,5,7,30,CLEAR,DUSK,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,NaN,IMPROPER LANE USAGE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.765601346906 41.880344977442),NaN,2.0
705198,a1f562015cbc6bfecec083ee97dad0cca3d9f770682764...,2023-07-06 22:45:00,22,5,7,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,IMPROPER TURNING/NO SIGNAL,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.633803891217 41.875532652069),NaN,4.0
708813,16ec144e4a39707b2d7c51fd924bc53dff5894de75d085...,2023-07-07 00:17:00,0,6,7,25,CLEAR,DARKNESS,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,UNABLE TO DETERMINE,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.705160226138 41.843575982562),NaN,2.0
708775,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,CLEAR,DAWN,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,NaN,FAILING TO YIELD RIGHT-OF-WAY,NO INDICATION OF INJURY,0.0,0.0,0.0,POINT (-87.763330444094 41.892623022942),NaN,3.0


In [74]:
df_master_agg_2018_2023.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601186 entries, 0 to 739104
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   CRASH_RECORD_ID          601186 non-null  object        
 1   CRASH_DATE               601186 non-null  datetime64[ns]
 2   CRASH_HOUR               601186 non-null  int64         
 3   CRASH_DAY_OF_WEEK        601186 non-null  int64         
 4   CRASH_MONTH              601186 non-null  int64         
 5   POSTED_SPEED_LIMIT       601186 non-null  int64         
 6   WEATHER_CONDITION        601186 non-null  object        
 7   LIGHTING_CONDITION       601186 non-null  object        
 8   ROADWAY_SURFACE_COND     601186 non-null  object        
 9   ROAD_DEFECT              601186 non-null  object        
 10  CRASH_TYPE               601186 non-null  object        
 11  HIT_AND_RUN_I            192710 non-null  object        
 12  PRIM_CONTRIBUTOR

In [75]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/df_master_agg_2018_2023.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_master_agg_2018_2023.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# [OLD] Re merging with just the records from people and vehicles that match on VEHICLE_ID, CRASH_UNIT_ID

In [30]:
# crashes and vehicles
# indicator=True creates an additional column to keep record of dataframe/table origin
# see https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4 for more info
df_master_v2 = pd.merge(crashes_edit_df, vehicles_edit_filtered_df,
                     on='CRASH_RECORD_ID', how='left', indicator=True)

In [31]:
# renaming _merge column before merging people
df_master_v2.rename(columns={"_merge": "merge_1_crashes_vehicles"}, inplace=True)

In [32]:
df_master_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1255640 entries, 0 to 1255639
Data columns (total 28 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   CRASH_RECORD_ID           1255640 non-null  object        
 1   CRASH_DATE_x              1255640 non-null  datetime64[ns]
 2   CRASH_HOUR                1255640 non-null  int64         
 3   CRASH_DAY_OF_WEEK         1255640 non-null  int64         
 4   CRASH_MONTH               1255640 non-null  int64         
 5   POSTED_SPEED_LIMIT        1255640 non-null  int64         
 6   WEATHER_CONDITION         1255640 non-null  object        
 7   LIGHTING_CONDITION        1255640 non-null  object        
 8   ROADWAY_SURFACE_COND      1255640 non-null  object        
 9   ROAD_DEFECT               1255640 non-null  object        
 10  CRASH_TYPE                1255640 non-null  object        
 11  HIT_AND_RUN_I             396888 non-null   object

In [33]:
# adding people
df_master_v2_step2 = pd.merge(df_master_v2, people_edit_filtered_df,
                     on='CRASH_RECORD_ID', how='left', indicator=True)

In [120]:
df_master_v2_step2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2619191 entries, 0 to 2619190
Data columns (total 44 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   CRASH_RECORD_ID           object        
 1   CRASH_DATE_x              datetime64[ns]
 2   CRASH_HOUR                int64         
 3   CRASH_DAY_OF_WEEK         int64         
 4   CRASH_MONTH               int64         
 5   POSTED_SPEED_LIMIT        int64         
 6   WEATHER_CONDITION         object        
 7   LIGHTING_CONDITION        object        
 8   ROADWAY_SURFACE_COND      object        
 9   ROAD_DEFECT               object        
 10  CRASH_TYPE                object        
 11  HIT_AND_RUN_I             object        
 12  PRIM_CONTRIBUTORY_CAUSE   object        
 13  MOST_SEVERE_INJURY        object        
 14  INJURIES_TOTAL            float64       
 15  INJURIES_FATAL            float64       
 16  INJURIES_INCAPACITATING   float64       
 17  LOCATION

In [34]:
# renaming second _merge column
df_master_v2_step2.rename(columns={"_merge": "merge_2_people" }, inplace=True)

In [35]:
# New aggregate columns

df_master_v2_step2['total_vehicles_in_crash'] = df_master_v2_step2.groupby('CRASH_RECORD_ID')['CRASH_UNIT_ID'].transform('count')
df_master_v2_step2['total_people_in_crash'] = df_master_v2_step2.groupby('CRASH_RECORD_ID')['PERSON_ID'].transform('count')

In [124]:
df_master_v2_step2.head()

,CRASH_RECORD_ID,CRASH_DATE_x,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people,total_vehicles_in_crash,total_people_in_crash
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,0.0,2.0,both,3,3
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,IMPROPER TURN,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,3,3
2,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,3,3
3,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,2018-09-05 08:40:00,8,4,9,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,FAILED TO YIELD,OTHER,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,4,4
4,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,2018-09-05 08:40:00,8,4,9,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,4,4


In [125]:
df_master_v2_step2.sort_values(by='CRASH_DATE')

,CRASH_RECORD_ID,CRASH_DATE_x,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people,total_vehicles_in_crash,total_people_in_crash
1552696,a802658be15312809c771559e4f81088cfb226830792a5...,2013-03-03 16:48:00,16,1,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,2.0,both,1,1
62094,19fb5af681f833c2af85734245f737fa6fbe62ac1ea379...,2013-06-01 20:29:00,20,7,6,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,...,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,1.0,both,2,2
62095,19fb5af681f833c2af85734245f737fa6fbe62ac1ea379...,2013-06-01 20:29:00,20,7,6,30,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,NO DEFECTS,...,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,1.0,both,2,2
2255850,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18 18:14:00,18,7,1,30,CLEAR,DUSK,DRY,NO DEFECTS,...,IMPROPER PARKING,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both,4,4
2255849,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18 18:14:00,18,7,1,30,CLEAR,DUSK,DRY,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,both,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618905,5583951a8ba8ee64a82c25ec60e57eeb8239391bc25c29...,2023-07-01 12:18:00,12,7,7,30,CLOUDY/OVERCAST,DAYLIGHT,DRY,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,0
2618974,8ba1b92e005cb5fc38bbb8705ad1ab6335b84a9060c1e2...,2023-07-01 16:30:00,16,7,7,30,RAIN,DAYLIGHT,WET,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,0
2619076,f69a8669c9a1eb22e188fbe568f7b281c888109dc1bfd0...,2023-07-01 15:20:00,15,7,7,20,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,0
2619115,d3173a5161ed13b40c8621798926a5f00b78832df693d8...,2023-07-01 15:20:00,15,7,7,10,OTHER,DAYLIGHT,DRY,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,0,0


In [36]:
# filtering by date
df_2018_2023_master_v2 = df_master_v2_step2[(df_master_v2_step2['CRASH_DATE'] > "2017-12-31 23:59:59")]

In [37]:
df_2018_2023_master_v2.columns

Index(['CRASH_RECORD_ID', 'CRASH_DATE_x', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'LOCATION', 'CRASH_UNIT_ID', 'CRASH_DATE_y',
       'UNIT_TYPE', 'VEHICLE_USE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I',
       'merge_1_crashes_vehicles', 'PERSON_ID', 'VEHICLE_ID', 'CRASH_DATE',
       'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
       'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT',
       'BAC_RESULT_VALUE', 'is_driver', 'total_drivers_in_crash',
       'merge_2_people', 'total_vehicles_in_crash', 'total_people_in_crash'],
      dtype='object')

In [38]:
# re-ordering columns, dropping duplicate 'CRASH_DATE' columns
selected_columns = ['CRASH_RECORD_ID', 'CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'POSTED_SPEED_LIMIT', 'EXCEED_SPEED_LIMIT_I','WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'LOCATION', 'CRASH_UNIT_ID',
       'UNIT_TYPE', 'VEHICLE_USE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'OCCUPANT_CNT',
       'merge_1_crashes_vehicles', 'PERSON_ID', 'VEHICLE_ID',
       'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
       'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT',
       'BAC_RESULT_VALUE', 'is_driver', 'total_drivers_in_crash', 'merge_2_people']
df_2018_2023_master_v2 = df_2018_2023_master_v2[selected_columns].copy()

In [42]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/df_2018_2023_master_v2.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_2018_2023_master_v2.to_csv(f)

#Previewing new master dataframe

In [43]:
df_2018_2023_master_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2080501 entries, 0 to 2619190
Data columns (total 42 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   CRASH_RECORD_ID           object        
 1   CRASH_DATE                datetime64[ns]
 2   CRASH_HOUR                int64         
 3   CRASH_DAY_OF_WEEK         int64         
 4   CRASH_MONTH               int64         
 5   POSTED_SPEED_LIMIT        int64         
 6   EXCEED_SPEED_LIMIT_I      object        
 7   WEATHER_CONDITION         object        
 8   LIGHTING_CONDITION        object        
 9   ROADWAY_SURFACE_COND      object        
 10  ROAD_DEFECT               object        
 11  CRASH_TYPE                object        
 12  HIT_AND_RUN_I             object        
 13  PRIM_CONTRIBUTORY_CAUSE   object        
 14  MOST_SEVERE_INJURY        object        
 15  INJURIES_TOTAL            float64       
 16  INJURIES_FATAL            float64       
 17  INJURIES

In [44]:
df_2018_2023_master_v2.head()

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,EXCEED_SPEED_LIMIT_I,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,...,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,NaN,CLEAR,DAYLIGHT,DRY,...,NaN,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,0.0,2.0,both
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,NaN,CLEAR,DAYLIGHT,DRY,...,REFUSED,"REPORTED, NOT EVIDENT",IMPROPER TURN,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both
2,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,NaN,CLEAR,DAYLIGHT,DRY,...,REFUSED,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both
3,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,2018-09-05 08:40:00,8,4,9,30,NaN,CLEAR,DAYLIGHT,DRY,...,NaN,NO INDICATION OF INJURY,FAILED TO YIELD,OTHER,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both
4,792b539deaaad65ee5b4a9691d927a34d298eb33d42af0...,2018-09-05 08:40:00,8,4,9,30,NaN,CLEAR,DAYLIGHT,DRY,...,NaN,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both


In [45]:
df_2018_2023_master_v2.sort_values(by='CRASH_DATE')

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,EXCEED_SPEED_LIMIT_I,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,...,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people
400404,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,0,2,1,25,NaN,UNKNOWN,UNKNOWN,UNKNOWN,...,DNA,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both
400405,0ad333cd371ec3976f9fac8160ccfc91f0399b48a92b7f...,2018-01-01 00:00:00,0,2,1,25,NaN,UNKNOWN,UNKNOWN,UNKNOWN,...,DNA,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both
752242,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,0,2,1,30,NaN,CLEAR,UNKNOWN,UNKNOWN,...,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both
752241,351626f25a540a0fccbe42fb815b14ae6c75c15676bf7c...,2018-01-01 00:00:00,0,2,1,30,NaN,CLEAR,UNKNOWN,UNKNOWN,...,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both
2275022,f798e1cdba8adfb80ee2aaeea541a55dd32fa8aff38494...,2018-01-01 00:00:00,0,2,1,20,NaN,CLEAR,DAYLIGHT,DRY,...,NaN,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544158,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,NaN,CLEAR,DAWN,DRY,...,NaN,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both
2544157,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,NaN,CLEAR,DAWN,DRY,...,NaN,NO INDICATION OF INJURY,FAILED TO YIELD,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both
2544159,6663caed20a5a0bcdb424989ce70bd1e4bca71beffd4ba...,2023-07-07 00:19:00,0,6,7,30,NaN,CLEAR,DAWN,DRY,...,NaN,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,NaN,0.0,2.0,both
2542351,c449bb5ee04a25261ec126d7ff1ee42cafce8e11ed9230...,2023-07-07 00:30:00,0,6,7,30,NaN,CLEAR,"DARKNESS, LIGHTED ROAD",DRY,...,NaN,NO INDICATION OF INJURY,OTHER,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,1.0,both


#Investigating dupes again

In [56]:
print(crashes_edit_df['CRASH_RECORD_ID'].duplicated().sum())
print(people_edit_filtered_df['CRASH_RECORD_ID'].duplicated().sum())
print(vehicles_edit_filtered_df['CRASH_RECORD_ID'].duplicated().sum())

0
767369
516535


In [46]:
df_2018_2023_master_v2['CRASH_RECORD_ID'].value_counts()

09aa8d6e2906482df6717671a532324d3667b00920b363a65face81089921576c5175803211ef1fc902662b7cb7df5e7268ab8491eb4fb4444a888fac30c888a    144
023f7de8d0f7e90dc072b7b88b62502f306469597481dabc2469aecc67a1026d485ba885476a3ea258a0df3cf6543b77b69cdbfabca44ab64b0fc64c00c808ef    143
1829f52c1281a0396ef94692331b3dc530bc4be5a54cd55e94c24a5e5e49b800fbcf9f24dabe4c8277c8964ad05aadc89e90fd94021959d6dff5fad55480d595    135
c9d233e31a4f2a07733ef75f0404e75c360b30c7ee9bc45076938dc80c375578c1468bc096ecb773d2bfc71270d746d95f416a5bd6b15fbcf8707b1748693722    130
7263be3b2bded65fb935c46628eb951fa2be8fa27fc6aa27d008ba17e99bc205e6a3023616216397f886f673d149e2462bc86844ab7ec61b1550cb8fd0b72bdd    120
                                                                                                                                   ... 
1f04887e2f4415b50ca6657d81a26353c3040d97f6ec1c70b39390c3448d3719033a5165a4737a38e6a2402500165521a675e3cf475fe43675d01fd8ad7a736d      1
1fb479ae362295d72c8387bcbd69715393576379b9bd2bda

In [47]:
df_check_v2 = df_2018_2023_master_v2.loc[df_2018_2023_master_v2['CRASH_RECORD_ID'] == '09aa8d6e2906482df6717671a532324d3667b00920b363a65face81089921576c5175803211ef1fc902662b7cb7df5e7268ab8491eb4fb4444a888fac30c888a']

In [48]:
df_check_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 2363234 to 2363377
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CRASH_RECORD_ID           144 non-null    object        
 1   CRASH_DATE                144 non-null    datetime64[ns]
 2   CRASH_HOUR                144 non-null    int64         
 3   CRASH_DAY_OF_WEEK         144 non-null    int64         
 4   CRASH_MONTH               144 non-null    int64         
 5   POSTED_SPEED_LIMIT        144 non-null    int64         
 6   EXCEED_SPEED_LIMIT_I      0 non-null      object        
 7   WEATHER_CONDITION         144 non-null    object        
 8   LIGHTING_CONDITION        144 non-null    object        
 9   ROADWAY_SURFACE_COND      144 non-null    object        
 10  ROAD_DEFECT               144 non-null    object        
 11  CRASH_TYPE                144 non-null    object        
 12  HIT_AND_RUN_

In [49]:
df_check_v2.head()

,CRASH_RECORD_ID,CRASH_DATE,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,EXCEED_SPEED_LIMIT_I,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,...,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people
2363234,09aa8d6e2906482df6717671a532324d3667b00920b363...,2022-11-23 17:00:00,17,4,11,30,NaN,CLEAR,DUSK,DRY,...,NaN,NONINCAPACITATING INJURY,OTHER,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,8.0,both
2363235,09aa8d6e2906482df6717671a532324d3667b00920b363...,2022-11-23 17:00:00,17,4,11,30,NaN,CLEAR,DUSK,DRY,...,UNIVERSITY OF CHICAGO,INCAPACITATING INJURY,OTHER,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,8.0,both
2363236,09aa8d6e2906482df6717671a532324d3667b00920b363...,2022-11-23 17:00:00,17,4,11,30,NaN,CLEAR,DUSK,DRY,...,NaN,FATAL,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,8.0,both
2363237,09aa8d6e2906482df6717671a532324d3667b00920b363...,2022-11-23 17:00:00,17,4,11,30,NaN,CLEAR,DUSK,DRY,...,CHRIST HOSPITAL,NONINCAPACITATING INJURY,OTHER,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,8.0,both
2363238,09aa8d6e2906482df6717671a532324d3667b00920b363...,2022-11-23 17:00:00,17,4,11,30,NaN,CLEAR,DUSK,DRY,...,NaN,NONINCAPACITATING INJURY,OTHER,UNKNOWN,UNKNOWN,TEST NOT OFFERED,NaN,1.0,8.0,both


In [51]:
#from google.colab import drive
#drive.mount('/content/drive')
path = '/content/drive/My Drive/df_check_v2.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_check_v2.to_csv(f)

In [52]:
df_check_v2['PERSON_ID'].value_counts()

O1468424    9
O1468427    9
O1468396    9
O1468397    9
O1468398    9
O1468399    9
O1468400    9
O1468436    9
P325277     9
P325280     9
P325281     9
P325283     9
P325284     9
P325285     9
P325287     9
P325288     9
Name: PERSON_ID, dtype: int64

In [53]:
df_check_v2['VEHICLE_ID'].value_counts()

1395675.0    36
1395681.0    27
1395647.0    18
1395657.0    18
1395665.0    18
1395672.0     9
1395652.0     9
1395671.0     9
Name: VEHICLE_ID, dtype: int64

In [54]:
df_check_v2['CRASH_UNIT_ID'].value_counts()

1468396.0    16
1468397.0    16
1468398.0    16
1468399.0    16
1468424.0    16
1468427.0    16
1468436.0    16
1468437.0    16
1468400.0    16
Name: CRASH_UNIT_ID, dtype: int64

In [55]:
df_check_v2['CRASH_RECORD_ID'].value_counts()

09aa8d6e2906482df6717671a532324d3667b00920b363a65face81089921576c5175803211ef1fc902662b7cb7df5e7268ab8491eb4fb4444a888fac30c888a    144
Name: CRASH_RECORD_ID, dtype: int64

##Another fail below

In [114]:
df_master = pd.merge(crashes_edit_df, people_edit_filtered_df, on='CRASH_RECORD_ID', how='left')
df_master = pd.merge(df_master, vehicles_edit_filtered_df,
                     left_on=['CRASH_RECORD_ID', 'VEHICLE_ID'],
                     right_on=['CRASH_RECORD_ID', 'CRASH_UNIT_ID'],
                     how='left')


In [115]:
# New aggregate columns

df_master['total_vehicles_in_crash'] = df_master.groupby('CRASH_RECORD_ID')['CRASH_UNIT_ID'].transform('count')
df_master['total_people_in_crash'] = df_master.groupby('CRASH_RECORD_ID')['PERSON_ID'].transform('count')

In [116]:
#OMG NEVERMIND
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1506474 entries, 0 to 1506473
Data columns (total 44 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   CRASH_RECORD_ID          1506474 non-null  object        
 1   CRASH_DATE_x             1506474 non-null  datetime64[ns]
 2   CRASH_HOUR               1506474 non-null  int64         
 3   CRASH_DAY_OF_WEEK        1506474 non-null  int64         
 4   CRASH_MONTH              1506474 non-null  int64         
 5   POSTED_SPEED_LIMIT       1506474 non-null  int64         
 6   WEATHER_CONDITION        1506474 non-null  object        
 7   LIGHTING_CONDITION       1506474 non-null  object        
 8   ROADWAY_SURFACE_COND     1506474 non-null  object        
 9   ROAD_DEFECT              1506474 non-null  object        
 10  CRASH_TYPE               1506474 non-null  object        
 11  HIT_AND_RUN_I            409570 non-null   object        
 12  

##Old merge with too many dupes below

In [38]:
# crashes and vehicles
# indicator=True creates an additional column to keep record of dataframe/table origin
# see https://towardsdatascience.com/left-join-with-pandas-data-frames-in-python-c29c85089ba4 for more info
df_master = pd.merge(crashes_edit_df, vehicles_edit_df,
                     on='CRASH_RECORD_ID', how='left', indicator=True)

In [39]:
# renaming _merge column before merging people
df_master.rename(columns={"_merge": "merge_1_crashes_vehicles"}, inplace=True)

In [40]:
# adding people
df_master_step2 = pd.merge(df_master, people_edit_df,
                     on='CRASH_RECORD_ID', how='left', indicator=True)

In [41]:
# renaming second _merge column
df_master_step2.rename(columns={"_merge": "merge_2_people" }, inplace=True)

In [52]:
# adding 2 more aggregate columns
df_master_step2['total_vehicles_in_crash'] = df_master_step2.groupby('CRASH_RECORD_ID')['CRASH_UNIT_ID'].transform('count')
df_master_step2['total_people_in_crash'] = df_master_step2.groupby('CRASH_RECORD_ID')['PERSON_ID'].transform('count')

In [53]:
df_master_step2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3403100 entries, 0 to 3403099
Data columns (total 46 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   CRASH_RECORD_ID           object        
 1   CRASH_DATE_x              datetime64[ns]
 2   CRASH_HOUR                int64         
 3   CRASH_DAY_OF_WEEK         int64         
 4   CRASH_MONTH               int64         
 5   POSTED_SPEED_LIMIT        int64         
 6   WEATHER_CONDITION         object        
 7   LIGHTING_CONDITION        object        
 8   ROADWAY_SURFACE_COND      object        
 9   ROAD_DEFECT               object        
 10  CRASH_TYPE                object        
 11  HIT_AND_RUN_I             object        
 12  PRIM_CONTRIBUTORY_CAUSE   object        
 13  MOST_SEVERE_INJURY        object        
 14  INJURIES_TOTAL            float64       
 15  INJURIES_FATAL            float64       
 16  INJURIES_INCAPACITATING   float64       
 17  LOCATION

In [54]:
df_master_step2.head()

,CRASH_RECORD_ID,CRASH_DATE_x,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people,total_vehicles_in_crash,total_people_in_crash
0,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,0.0,2.0,both,6,6
1,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,IMPROPER TURN,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,6,6
2,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,6,6
3,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,NaN,NaN,NaN,NaN,NaN,0.0,2.0,both,6,6
4,79c7a2ce89f446262efd86df3d72d18b04ba487024b7c4...,2019-03-25 14:43:00,14,2,3,30,CLEAR,DAYLIGHT,DRY,NO DEFECTS,...,IMPROPER TURN,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,2.0,both,6,6


# [OLD] Invesitgating why one crash would have so many records

In [55]:
df_master_step2['CRASH_RECORD_ID'].value_counts()

023f7de8d0f7e90dc072b7b88b62502f306469597481dabc2469aecc67a1026d485ba885476a3ea258a0df3cf6543b77b69cdbfabca44ab64b0fc64c00c808ef    192
c9d233e31a4f2a07733ef75f0404e75c360b30c7ee9bc45076938dc80c375578c1468bc096ecb773d2bfc71270d746d95f416a5bd6b15fbcf8707b1748693722    156
74e9d893f1df817db0fd24a7d10006941f1eda0c9e7a33c62bb2850e5a9560099b88bececab94471325d467f79f04ddf70ad92357f500c1e5765905d95278524    154
09aa8d6e2906482df6717671a532324d3667b00920b363a65face81089921576c5175803211ef1fc902662b7cb7df5e7268ab8491eb4fb4444a888fac30c888a    153
45ef99d6739c22de4260bc091e70dd495a1fd4d57557f56b05e53eff8cceaf39b714407d49866e64569dab903013eed067c1b1e13dee641f6b2ebfda3f745dc3    144
                                                                                                                                   ... 
b636d144a8ca405d7867889721fda4f8a529859713ff9b4c413c3bfc2d4dd82639a431fe5f5532972543637bac05fe2972bf0dd83b9fc5d6c891d6241f6bcb01      1
078ce231aa8b6a5f4d045494db9ef2b9ab520c767d084509

In [60]:
print(df_master_step2.loc[df_master_step2['CRASH_RECORD_ID'] == '023f7de8d0f7e90dc072b7b88b62502f306469597481dabc2469aecc67a1026d485ba885476a3ea258a0df3cf6543b77b69cdbfabca44ab64b0fc64c00c808ef'])

                                           CRASH_RECORD_ID  \
3169718  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169719  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169720  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169721  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169722  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
...                                                    ...   
3169905  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169906  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169907  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169908  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   
3169909  023f7de8d0f7e90dc072b7b88b62502f306469597481da...   

               CRASH_DATE_x  CRASH_HOUR  CRASH_DAY_OF_WEEK  CRASH_MONTH  \
3169718 2019-10-05 22:08:00          22                  7           10   
3169719 2019-10-05 22:08:00          22                  7           10   
3169720 2019-10-05 22:08:00   

In [62]:
df_check = df_master_step2.loc[df_master_step2['CRASH_RECORD_ID'] == '023f7de8d0f7e90dc072b7b88b62502f306469597481dabc2469aecc67a1026d485ba885476a3ea258a0df3cf6543b77b69cdbfabca44ab64b0fc64c00c808ef']

In [63]:
df_check.head()

,CRASH_RECORD_ID,CRASH_DATE_x,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people,total_vehicles_in_crash,total_people_in_crash
3169718,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169719,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,TOO FAST FOR CONDITIONS,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169720,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169721,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169722,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,TOO FAST FOR CONDITIONS,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192


In [64]:
df_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 3169718 to 3169909
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   CRASH_RECORD_ID           192 non-null    object        
 1   CRASH_DATE_x              192 non-null    datetime64[ns]
 2   CRASH_HOUR                192 non-null    int64         
 3   CRASH_DAY_OF_WEEK         192 non-null    int64         
 4   CRASH_MONTH               192 non-null    int64         
 5   POSTED_SPEED_LIMIT        192 non-null    int64         
 6   WEATHER_CONDITION         192 non-null    object        
 7   LIGHTING_CONDITION        192 non-null    object        
 8   ROADWAY_SURFACE_COND      192 non-null    object        
 9   ROAD_DEFECT               192 non-null    object        
 10  CRASH_TYPE                192 non-null    object        
 11  HIT_AND_RUN_I             0 non-null      object        
 12  PRIM_CONTRIB

###Exporting record check to csv

In [65]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/df_check.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_check.to_csv(f)

Mounted at /content/drive


##Continuing to investigate

In [66]:
df_check['PERSON_ID'].nunique()

16

In [67]:
df_check['VEHICLE_ID'].nunique()

6

In [80]:
df_O754015_p = people_edit_df.loc[people_edit_df['PERSON_ID'] == 'O754015']
df_O754015_p

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,PERSON_TYPE,SEX,AGE,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash
86803,O754015,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,716258.0,2019-10-05 10:00:00,DRIVER,F,NaN,MERCY,INCAPACITATING INJURY,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1,6


In [81]:
df_716258_p = people_edit_df.loc[people_edit_df['VEHICLE_ID'] == 716258.0]
df_716258_p

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,PERSON_TYPE,SEX,AGE,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash
86803,O754015,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,716258.0,2019-10-05 10:00:00,DRIVER,F,NaN,MERCY,INCAPACITATING INJURY,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1,6


In [82]:
df_716258_v = vehicles_edit_df.loc[vehicles_edit_df['CRASH_UNIT_ID'] == 716258.0]
df_716258_v

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_TYPE,VEHICLE_USE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I
775507,716258.0,3a328d0515dd14fe024c4fcfe6620db0836b530f6c3df5...,2019-08-13 17:23:00,DRIVER,PERSONAL,NONE,S,STRAIGHT AHEAD,1.0,NaN


In [85]:
df_716258_p = people_df.loc[people_df['VEHICLE_ID'] == 716258.0]
df_716258_p

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT_VALUE,CELL_PHONE_USE
86803,O754015,DRIVER,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,JC461568,716258.0,10/05/2019 10:00:00 AM,NaN,CHICAGO,IL,60653,...,NaN,FAILED TO YIELD,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [84]:
df_O754015_check = df_check.loc[df_check['PERSON_ID'] == 'O754015']
df_O754015_check

,CRASH_RECORD_ID,CRASH_DATE_x,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,POSTED_SPEED_LIMIT,WEATHER_CONDITION,LIGHTING_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,...,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash,merge_2_people,total_vehicles_in_crash,total_people_in_crash
3169718,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169734,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169750,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169766,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169782,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169798,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169814,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169830,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169846,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192
3169862,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,22,7,10,30,RAIN,"DARKNESS, LIGHTED ROAD",WET,NO DEFECTS,...,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1.0,6.0,both,192,192


In [86]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/df_O754015_check.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df_O754015_check.to_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
df_754016_v = vehicles_edit_df.loc[vehicles_edit_df['CRASH_UNIT_ID'] == 754016.0]
df_754016_v

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_TYPE,VEHICLE_USE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I
77728,754016.0,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 10:00:00,DRIVER,PERSONAL,NONE,N,STRAIGHT AHEAD,4.0,NaN


In [106]:
df_762332_v = vehicles_edit_df.loc[vehicles_edit_df['CRASH_UNIT_ID'] == 762332.0]
df_762332_v

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_TYPE,VEHICLE_USE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I
77943,762332.0,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,2019-10-05 22:08:00,PARKED,NOT IN USE,NONE,N,PARKED,0.0,NaN


In [107]:
df_762332_p = people_edit_df.loc[people_edit_df['VEHICLE_ID'] == 762332.0]
df_762332_p

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,PERSON_TYPE,SEX,AGE,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash
1151748,O802813,d1710e318a8fd67f341b7077751d2c4f044dc55a3b606a...,762332.0,2019-12-15 15:05:00,DRIVER,F,30.0,REFUSED,NO INDICATION OF INJURY,NONE,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1,2


In [109]:
df_716258_v = vehicles_edit_df[vehicles_edit_df['CRASH_UNIT_ID'] == 716258.0]
df_716258_v

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_TYPE,VEHICLE_USE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I
775507,716258.0,3a328d0515dd14fe024c4fcfe6620db0836b530f6c3df5...,2019-08-13 17:23:00,DRIVER,PERSONAL,NONE,S,STRAIGHT AHEAD,1.0,NaN


In [110]:
df_716258_p = people_edit_df.loc[people_edit_df['VEHICLE_ID'] == 716258.0]
df_716258_p

,PERSON_ID,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,PERSON_TYPE,SEX,AGE,HOSPITAL,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT_VALUE,is_driver,total_drivers_in_crash
86803,O754015,023f7de8d0f7e90dc072b7b88b62502f306469597481da...,716258.0,2019-10-05 10:00:00,DRIVER,F,NaN,MERCY,INCAPACITATING INJURY,FAILED TO YIELD,UNKNOWN,NORMAL,TEST NOT OFFERED,NaN,1,6


##End investigation


#[OLD]Filtering by date for master dataset

In [46]:
# filtering by date
df_2018_2023_master = df_master_step2[(df_master_step2['CRASH_DATE'] > "2017-12-31 23:59:59")]


In [49]:
df_2018_2023_master.columns

Index(['CRASH_RECORD_ID', 'CRASH_DATE_x', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'LOCATION', 'CRASH_UNIT_ID', 'CRASH_DATE_y',
       'UNIT_TYPE', 'VEHICLE_USE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I',
       'merge_1_crashes_vehicles', 'PERSON_ID', 'VEHICLE_ID', 'CRASH_DATE',
       'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
       'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT',
       'BAC_RESULT_VALUE', 'is_driver', 'total_drivers_in_crash',
       'merge_2_people'],
      dtype='object')

In [50]:
# re-ordering columns, dropping duplicate 'CRASH_DATE' columns
selected_columns = ['CRASH_RECORD_ID', 'CRASH_DATE', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'POSTED_SPEED_LIMIT', 'EXCEED_SPEED_LIMIT_I','WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'LOCATION', 'CRASH_UNIT_ID',
       'UNIT_TYPE', 'VEHICLE_USE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'OCCUPANT_CNT',
       'merge_1_crashes_vehicles', 'PERSON_ID', 'VEHICLE_ID',
       'PERSON_TYPE', 'SEX', 'AGE', 'HOSPITAL', 'INJURY_CLASSIFICATION',
       'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT',
       'BAC_RESULT_VALUE', 'is_driver', 'total_drivers_in_crash', 'merge_2_people']
df_2018_2023_master = df_2018_2023_master[selected_columns].copy()